In [ ]:
%gui qt

# open and show data (convert it to h5 to be loaded)

## open the Pilatus1M dataset

![hdf5treeView](img/hdf5_data.png)

In [ ]:
# from h5
dataPath="data/Pilatus1M.hdf5"
import h5py
import numpy
#select the cube values:
f=h5py.File(dataPath)
# then select the 'Data/qspace' datagroup (array containing the cubes of the isosurface )
data=f['data']
data = numpy.array(data, order='C', dtype='float32')

In [ ]:
# play with hdf5
from silx.gui.hdf5 import Hdf5TreeView
viewer=Hdf5TreeView()
viewer.findHdf5TreeModel().insertFile(dataPath)
viewer.show()

## Plot the data
 - using log scale
 
![data displayed by silx.gui.plot.Plot2D](img/pilatus_data.png)

In [ ]:
# using silx
from silx.gui.plot import Plot2D
plot=Plot2D()
plot.setKeepDataAspectRatio(True)
colormap = {
    'name': 'inferno',             
    'normalization': 'log',  
    'autoscale': True,       
    'vmin': 0.0,             
    'vmax': 1.0              
 }
plot.setDefaultColormap(colormap)
plot.addImage(data)
plot.show()

# compute radii to center for each pixels

In [ ]:
def computeradius(data, xcenter, ycenter):
    import numpy
    # do the azimutal integration
    xcenter=180
    ycenter=260
    r=numpy.zeros(data.shape)
#    y, x=numpy.ogrid[:data.shape[0], :data.shape[1]]
#    r=numpy.sqrt((x-xcenter)**2+(y-ycenter)**2)
    for y in range(data.shape[0]):
        for x in range(data.shape[1]):
            r[y,x]=numpy.sqrt(numpy.square(x-xcenter)+numpy.square(y-ycenter))
    return r

In [ ]:
# V2
import numpy
def computeradius(data, xcenter, ycenter):
    
    # do the azimutal integration
    xcenter=180
    ycenter=260
    y, x=numpy.ogrid[:data.shape[0], :data.shape[1]]
    r=numpy.sqrt((x-xcenter)**2+(y-ycenter)**2)
    return r

In [ ]:
# do the azimutal integration
radii=computeradius(data, xcenter=180, ycenter=260)
# if plotted : increase the number of voxel : logical 

# create the histogram of the radii

- histo_range=[0, int(numpy.ceil(radii.max()))]

In [ ]:
from silx.math import histogram
nbbins=int(numpy.ceil(radii.max()))
histo_range=[0,int(numpy.ceil(radii.max()))]
histo, w_histo, edges = histogram.Histogramnd(radii.ravel(), n_bins=nbbins, histo_range=histo_range)

# plot the histogram

In [ ]:
from silx.gui.plot import Plot1D 
p=Plot1D()
p.addCurve(x=range(int(numpy.ceil(radii.max()))), y=histo, legend='colorDistribution')
p.show()

# azimutal integration using weights

- A simplify version is to get the mean contribution of each pixels through radii


In [ ]:
from silx.math import histogram
nbbins=int(numpy.ceil(radii.max()))
histo_range=[0,int(numpy.ceil(radii.max()))]
histo, w_histo, edges = histogram.Histogramnd(radii.ravel(), weights=data.ravel(), n_bins=nbbins, histo_range=histo_range)
binscenter=(edges[0][1:] + edges[0][0:-1]) / 2.0

In [ ]:
p=Plot1D()
p.addCurve(x=binscenter, y=histo, legend='unweighted',color='red')
p.addCurve(x=binscenter, y=w_histo, legend='weighted',color='blue')
p.addCurve(x=binscenter, y=w_histo/histo, legend='integration',color='green')
p.setActiveCurve(None)
p.show()

In [ ]:
# the same with numpy.histogram
%timeit histoNumpy=numpy.histogram(radii.ravel(), nbbins, weights=data.ravel())
binscenter=(histoNumpy[1][1:]+histoNumpy[1][0:-1])/2.0